### Student Information
Name: 李享 Louis

Student ID: 111062699

GitHub ID:

Kaggle name:

Kaggle private scoreboard snapshot:

[Snapshot](img/pic0.png)

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2022-Lab2-master Repo](https://github.com/keziatamus/DM2022-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm2022-isa5810-lab2-homework) regarding Emotion Recognition on Twitter by this link https://www.kaggle.com/t/2b0d14a829f340bc88d2660dc602d4bd. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission __BEFORE the deadline (Nov. 22th 11:59 pm, Tuesday)_. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook** and **add minimal comments where needed**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 25th 11:59 pm, Friday)__. 

In [ ]:
### Begin Assignment Here


import numpy as np
import pandas as pd

path = 'dm2022-isa5810-lab2-homework/'
ori_tweets_data = pd.read_json(path + 'tweets_DM.json', lines=True)
ori_tweets_label = pd.read_csv(path + 'emotion.csv')
tweet_train_predict_split = pd.read_csv(path + 'data_identification.csv')

In [ ]:
# '_index' and '_type' are all the same in this dataset, so drop it.

tweets_data = ori_tweets_data.drop(columns=['_index', '_type'])
tweets_label = ori_tweets_label.drop(columns=['tweet_id'])

In [ ]:
import json
j = json.loads(json.dumps(tweets_data['_source'].values[0], indent=4))['tweet']
# all informations are in `tweet` key, so I need to extract all informations in `tweet`
print(j)

In [ ]:
import re

score, hashtags, tweet_id, text = [], [], [], []
score = pd.DataFrame({ 'score': tweets_data['_score'].values.tolist() })

for idx, d in enumerate(tweets_data['_source'].values):
    if len(d['tweet']['hashtags']) == 0:
        hashtags.append(None)
    else:
        hashtags.append(d['tweet']['hashtags'])
    tweet_id.append(d['tweet']['tweet_id'])
    text.append(re.sub(r'[^\w\s]', '', d['tweet']['text']))

hashtags = pd.DataFrame({'hashtags': hashtags})
tweet_id = pd.DataFrame(tweet_id, columns=['tweet_id'])
text = pd.DataFrame(text, columns=['text'])

tweets_data = pd.concat([tweets_data, hashtags, tweet_id, text], axis=1)
tweets_data = tweets_data.drop(columns=['_source'])
# tweets_data[:5]

In [ ]:
# tweets_data.to_csv(path + 'split_tweet_data.csv', index=False)
split_data = pd.read_csv(path + 'split_tweet_data.csv', lineterminator='\n')

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# tfidf = TfidfVectorizer()
# X = tfidf.fit_transform(split_data['text'])
# X.shape # (1867535, 933053)

In [ ]:
emotions = pd.read_csv(path + 'emotion.csv')

# set(emotions['emotion'])

sadness = emotions.loc[emotions['emotion'] == 'sadness']
disgust = emotions.loc[emotions['emotion'] == 'disgust']
anticipation = emotions.loc[emotions['emotion'] == 'anticipation']
joy = emotions.loc[emotions['emotion'] == 'joy']
anger = emotions.loc[emotions['emotion'] == 'anger']
fear = emotions.loc[emotions['emotion'] == 'fear']
surprise = emotions.loc[emotions['emotion'] == 'surprise']
trust = emotions.loc[emotions['emotion'] == 'trust']

In [ ]:
df_emotion = pd.DataFrame(
    [
        len(sadness), len(disgust), len(anticipation), len(joy), len(anger), len(fear), len(surprise), len(trust)
    ],
    [
        'sadness', 'disgust', 'anticipation', 'joy', 'anger', 'fear', 'surprise', 'trust'
    ]
    , columns=['count'])
display(df_emotion)

### HuggingFace Bert testing.
I use HuggingFace as support in this homework, and I follow this link.
https://huggingface.co/docs/transformers/index

this is HuggingFace's official tutorial.

In [ ]:
from transformers import AutoTokenizer

In this homework, I use HuggingFace's "bert-base-uncased" pre-trained model.
You can find all pre-trained model on this link.

https://huggingface.co/models

Notice that the official document tell us `from_pretrained()` method uses `torch.load()`.

In [ ]:
train_indexes = tweet_train_predict_split.loc[tweet_train_predict_split['identification'] == 'train'].index
predict_indexes = tweet_train_predict_split.loc[tweet_train_predict_split['identification'] == 'test'].index

train_data_df = pd.DataFrame(split_data.iloc[train_indexes]['text'])
predict_data_df = pd.DataFrame(split_data.iloc[predict_indexes]['text'])

train_data_df.reset_index(drop=True, inplace=True)
tweets_label.reset_index(drop=True, inplace=True)

train_data_df = pd.concat([train_data_df, tweets_label], axis=1)
train_data_df[:3]

this part, I want to split my dataset into `train dataset` and `validation dataset`, and I have some problem here

In [ ]:
def datasets_split(dataset, ratio):
    length = len(dataset)
    shuffle_index = np.random.permutation(length)
    test_size = int(length * ratio)
    train_data = dataset.iloc[shuffle_index[test_size:]]
    test_data = dataset.iloc[shuffle_index[:test_size]]

    return train_data, test_data

X_train, X_test = datasets_split(train_data_df, 0.2)

X_train.reset_index(drop=True, inplace=True)
X_test.reset_index(drop=True, inplace=True)

# Although I only change column name `emotion` to `label`,
# but it can't work with X_train.columns.values[1] = 'label',
# and I can't find the reason
X_train.columns = ['text', 'label']
X_test.columns = ['text', 'label']

# I want to store my training and test dataset into csv file, and read them use `load_dataset()` later,
# it store successfully.
X_train.to_csv('selfmade-datasets/train.csv', index=False, line_terminator='\n')
X_test.to_csv('selfmade-datasets/test.csv', index=False, line_terminator='\n')

In [ ]:
import datasets
from datasets import Dataset
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')

encoder.fit(np.array(X_train['label']).reshape(-1, 1))
encoder.categories_
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # choose pre-trained model

def tokenizer_method(datas):
    output = tokenizer(datas["text"], padding="max_length", truncation=True)
    output['label'] = encoder.transform(np.array(datas['label']).reshape(-1, 1)).toarray()
    return output


dataset = datasets.DatasetDict(
    {
        'train': Dataset.from_pandas(X_train),
        'test': Dataset.from_pandas(X_test),
    }
)

# print(dataset['train'])
# print(dataset['train'][1])
# print(dataset['train']['text'][:5])

tokenized_dataset = dataset.map(tokenizer_method, batched=True)


In [ ]:
LABEL_COUNT = len(encoder.categories_[0])
# print(tokenized_dataset['train'][1]['token_type_ids'], end='\n\n')
# print(tokenized_dataset['train'][1]['attention_mask'], end='\n\n')
# print(tokenized_dataset['train'][1]['label'], end='\n\n')
# print(tokenized_dataset['train'][1]['input_ids'])

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
from transformers import AutoModelForSequenceClassification
import torch

model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=LABEL_COUNT)


In [ ]:
from transformers import TrainingArguments, Trainer
import evaluate

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

metric = evaluate.load("accuracy")
training_args = TrainingArguments(
    output_dir="training_result",
    evaluation_strategy="epoch",
    per_device_train_batch_size=64,
    num_train_epochs=1,
    save_steps=100000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test'],
    compute_metrics=compute_metrics,
)


In [ ]:
trainer.train()

OverLapping occur here, so I want to do underlapping and check the result.

In [ ]:
# data overlapping occured here, so I want to do
min_emotion_cnt = int(df_emotion.min())

In [ ]:
# reuse `train_data_df` variable from upper cell

sadness_sample = train_data_df.loc[train_data_df['emotion'] == 'sadness'].sample(n=min_emotion_cnt)
disgust_sample = train_data_df.loc[train_data_df['emotion'] == 'disgust'].sample(n=min_emotion_cnt)
anticipation_sample = train_data_df.loc[train_data_df['emotion'] == 'anticipation'].sample(n=min_emotion_cnt)
joy_sample = train_data_df.loc[train_data_df['emotion'] == 'joy'].sample(n=min_emotion_cnt)
anger_sample = train_data_df.loc[train_data_df['emotion'] == 'anger'].sample(n=min_emotion_cnt)
fear_sample = train_data_df.loc[train_data_df['emotion'] == 'fear'].sample(n=min_emotion_cnt)
surprise_sample = train_data_df.loc[train_data_df['emotion'] == 'surprise'].sample(n=min_emotion_cnt)
trust_sample = train_data_df.loc[train_data_df['emotion'] == 'trust'].sample(n=min_emotion_cnt)

In [ ]:
sampling_train_data_df = pd.concat([
    sadness_sample,
    disgust_sample,
    anticipation_sample,
    joy_sample,
    anger_sample,
    fear_sample,
    surprise_sample,
    trust_sample
]).sample(frac=1).reset_index(drop=True)

sampling_train_data_df = sampling_train_data_df.rename(columns={ 'emotion': 'label' })
display(sampling_train_data_df)

In [ ]:
sampling_train_data_df.to_csv('undersampling_datasets.csv', index=False, line_terminator='\n')

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(handle_unknown='ignore')
encoder.fit(np.array(sampling_train_data_df['label']).reshape(-1, 1))
SAMPLE_LABEL_COUNT = len(encoder.categories_[0])

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased") # choose pre-trained model

def tokenizer_method(datas):
    output = tokenizer(datas["text"], padding="max_length", truncation=True)
    output['label'] = encoder.transform(np.array(datas['label']).reshape(-1, 1)).toarray()
    return output

import datasets
from datasets import Dataset

sampling_dataset = datasets.DatasetDict(
    {
        'train': Dataset.from_pandas(sampling_train_data_df),
    }
)

sampling_tokenized_dataset = sampling_dataset.map(tokenizer_method, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=SAMPLE_LABEL_COUNT)

from transformers import TrainingArguments, Trainer
import evaluate

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

metric = evaluate.load("accuracy")
sampling_training_args = TrainingArguments(
    output_dir="sampling_training_result",
    eval_accumulation_steps="epoch",
    per_device_eval_batch_size=64,
    num_train_epochs=1,
    save_steps=100000,
)

sampling_trainer = Trainer(
    model=model,
    args=sampling_training_args,
    train_dataset=sampling_tokenized_dataset['train'],
    compute_metrics=compute_metrics,
)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
torch.cuda.set_device('cuda:0')

In [ ]:
sampling_trainer.train()